## Xbot2 ROS API

The most convenient way to interact with a Xbot2 robot in joint space is probably via the ROS middleware.
The XBot2 framework integrates with ROS via two plugins that must be included in its configuration file, 
namely:
 - `ros_io`, which handles (i) broadcasting of the robot state to ROS, (ii) plugin lifecycle management, (iii) safety and filtering
 - `ros_control` which forward any reference received on the */xbotcore/command* topic to the motors
 
Even though it is definetely possible to "manually" subscribe and publish to Xbot2's topics, a less error-prone and high level abstraction is provided, as well. Python bindings to the *XbotInterface* C++ package allow to control the robot via ROS with the same API that we have seen in all our plugin examples.

To start, let's import all required modules. **Remember to use Python2.7 on Ubuntu Xenial and Bionic, whereas Python3.X should be used on Focal.**

In [27]:
from xbot_interface import xbot_interface as xbot
from xbot_interface import config_options as opt
from cartesian_interface.affine3 import Affine3

import rospy
import numpy as np

rospy.init_node('xbot2_ros_tutorial')

### Creating a configuration object
Our first step is to create a configuration object, that allow the *RobotInterface* and *ModelInterface* classes to self-configure. This can be done from ROS parameters with the following function.

In [39]:
def get_xbot_cfg(is_fb):
    """
    A function to construct the xbotinterface config object from ros
    """
    urdf = rospy.get_param('/xbotcore/robot_description')
    srdf = rospy.get_param('/xbotcore/robot_description_semantic')
    
    cfg = opt.ConfigOptions()
    cfg.set_urdf(urdf)
    cfg.set_srdf(srdf)
    cfg.generate_jidmap()
    cfg.set_bool_parameter('is_model_floating_base', is_fb)
    cfg.set_string_parameter('model_type', 'RBDL')
    cfg.set_string_parameter('framework', 'ROS')
    
    return cfg

### Querying joint information
We can now create a class representing our robot, i.e. the *RobotInterface* and, e.g., print the list of joint names and joint positions.

In [40]:
cfg = get_xbot_cfg(is_fb=False)  # get config object
robot = xbot.RobotInterface(cfg)  # create robot (note: xbot2 should be up and running)

jnames = robot.getEnabledJointNames()   # get list of joint names
jpos = robot.getJointPosition()  # get actual joint position
jref = robot.getPositionReference()  # get actual position reference

for n, q, qref in zip(jnames, jpos, jref):
    print('{}: {} vs {}'.format(n, q, qref))

joint1: -0.000669108179864 vs -0.000658228527755
joint2: 1.09653973579 vs 1.06472229958
joint3: 1.03502476215 vs 1.02368366718
joint4: -8.23631562525e-05 vs -8.22785659693e-05
joint5: 0.0202679000795 vs 0.0137048419565


### Querying kinematics and dynamics
Kinematics and dynamics information is also available via the *ModelInterface* class.

In [44]:
robot.sense()  # update robot pose
model = robot.model()  # this model is automatically kept in sync with robot
Tee = model.getPose('TCP')
print('end effector pose w.r.t. world frame is:\n{}'.format(Tee))

end effector pose w.r.t. world frame is:
translation: [    0.7429, -0.0004976,    0.03149]
rotation   : [ 0.0002587,       0.88, -0.0001793,     0.4749]


### Interacting with plugins
We can interact with each plugin lifecycle with ROS services. In particular, each plugin defines:
  - a `switch` service that turns the plugin on or off
  - a `state` service returning the current plugin status
  - an `abort` service to force the plugin to stop
  
As an example, we will write some Python code that triggers a homing based on whether the joints are too far away from their homing position.

First let us create the required service clients, then write our simple logic.

In [48]:
# create ros service client
from std_srvs.srv import *
homing_switch = rospy.ServiceProxy('/xbotcore/homing/switch', SetBool)

# get the home state from srdf
qhome = robot.getRobotState('home')

# update robot and get its current position
robot.sense()
q = robot.getJointPosition()

# do homing if needed
if np.linalg.norm(qhome - q) > 0.1:
    print('homing triggered')
    homing_switch(True)
else:
    print('homing not needed')
    

homing not needed


### Interacting with the robot joints
Let's see how to send references to the robot joints. For the purpose of this example, we will send a sinusoidal trajectory to the joints, together with a computed torque feedforward.

First of all, it is necessary to enable the *ros_control* plugin. Note that is has been included in the configuration file under the *ros_ctrl* name.

In [54]:
ros_ctrl_switch = rospy.ServiceProxy('/xbotcore/ros_ctrl/switch', SetBool)
ros_ctrl_switch(True)

success: True
message: "Successfully started 'ros_ctrl'"

In [56]:
import math

# trajectory parameters
q_range = 0.1  # sine amplitude, [rad]
period = 2.0  # sine period
omega = 2*math.pi/period  # sine omega

# get starting position
robot.sense()
q0 = robot.getJointPosition()

# looping utils
time = 0.0  # current time since trj start
dt = 0.01  # control dt (100 Hz)
rate = rospy.Rate(1./dt) # rate object to synchronize the loop

print('Trajectory started!')

while time < 5*period:
    
    # compute trajectory point
    q = q0*0.5*(math.cos(omega*time) + 1)
    v = -q0*0.5*omega*math.sin(omega*time)
    a = -q0*0.5*omega**2*math.cos(omega*time)
    
    # compute inverse dynamics torque
    model.setJointPosition(q)
    model.setJointVelocity(v)
    model.setJointAcceleration(a)
    model.update()
    tau = model.computeInverseDynamics()
    
    # set reference to robot
    robot.setPositionReference(q)
    robot.setVelocityReference(v)
    robot.setEffortReference(tau)
    
    # move() does the publishing to /xbotcore/command
    robot.move()
    
    # sync the loop to the target frequency
    time += dt
    rate.sleep()

print('Trajectory ended!')

Trajectory started!
Trajectory ended!


### Control modes
By default, invoking `robot.move()` will send a **complete reference message** to the robot, i.e. containing a target position, velocity, torque, stiffness, and damping gains.

This is rarely optimal. If, for instance, we only want to send position and effort to the joints, we need to modify the robot's contol mode as follows.

In [50]:
# set a default control mode (applied to all joints)
default_ctrl_mode = xbot.ControlMode.Position()
robot.setControlMode(default_ctrl_mode)

# override control mode for a single joint
ctrl_mode_override = {
    'joint1': xbot.ControlMode.Position() + xbot.ControlMode.Effort()
}
robot.setControlMode(ctrl_mode_override)

True

Repeating the execution of the sinusoidal trajectory will send the torque reference only to the first joint this time.